## 1 Неработающая нейросеть

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Пример данных
data = {
    "description": [
        "Apple iPhone 13 with 128GB storage",
        "Samsung Galaxy S21 Smartphone",
        "Organic cotton T-shirt size M",
        "Wireless Bluetooth Headphones",
        "Stainless steel kitchen knife set",
        "Leather office chair black"
    ],
    "category": [
        "electronics",
        "electronics",
        "clothing",
        "electronics",
        "kitchen",
        "furniture"
    ]
}

df = pd.DataFrame(data)

# Кодируем целевые метки
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df["category"])
y_categorical = to_categorical(y_encoded)

# Разделяем на train и test
X_train, X_test, y_train, y_test = train_test_split(
    df["description"], y_categorical, test_size=0.3, random_state=42)

# Токенизация текстов
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Приведение последовательностей к одной длине
max_len = 20
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# Построение модели нейросети
model = Sequential([
    Embedding(input_dim=1000, output_dim=64, input_length=max_len),
    GlobalAveragePooling1D(),
    Dense(32, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(X_train_pad, y_train, epochs=20, batch_size=4, verbose=1)

# Оценка модели
loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test accuracy: {accuracy:.4f}")

# Пример предсказания
texts = ["New wireless mouse", "Blue denim jacket"]
seqs = tokenizer.texts_to_sequences(texts)
pads = pad_sequences(seqs, maxlen=max_len, padding='post')
preds = model.predict(pads)
pred_labels = label_encoder.inverse_transform(preds.argmax(axis=1))
for text, label in zip(texts, pred_labels):
    print(f"'{text}' → predicted category: {label}")


# 2 Работающий простой пример

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE  # Для балансировки классов
from imblearn.over_sampling import RandomOverSampler 

In [3]:
data = pd.read_excel('tmc.xlsx')

In [9]:

# Функция для обучения нескольких моделей и выбора лучшей с учетом дисбаланса
def train_and_select_best_model(X_train, y_train, X_test, y_test):
    models = {
        "Logistic Regression": LogisticRegression(multi_class="ovr", max_iter=1000, random_state=42, class_weight='balanced'),
        "Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced'),
        "Linear SVM": LinearSVC(max_iter=1000, random_state=42, class_weight='balanced')
    }
    
    best_model = None
    best_accuracy = 0
    best_model_name = ""
    
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        print(f"Модель: {name} | Точность: {acc:.3f}")
        print(classification_report(y_test, y_pred))
        
        if acc > best_accuracy:
            best_accuracy = acc
            best_model = model
            best_model_name = name
    
    print(f"Лучшая модель: {best_model_name} с точностью {best_accuracy:.3f}")
    return best_model



# Пример данных

df = pd.DataFrame(data)

X = df["description"]
y = df["category"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Решение проблемы дисбаланса с помощью SMOTE
smote = SMOTE(random_state=42, k_neighbors=3)
ros = RandomOverSampler(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_vect, y_train)
X_train_bal, y_train_bal = ros.fit_resample(X_train_vect, y_train)
best_model = train_and_select_best_model(X_train_smote, y_train_smote, X_test_vect, y_test)






c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Модель: Logistic Regression | Точность: 0.880
                                           precision    recall  f1-score   support

                                 Абразивы       1.00      0.80      0.89         5
                    Адаптеры компьютерные       0.56      1.00      0.71         5
                                  Антенны       1.00      1.00      1.00         9
           Аппараты телефонные (телефоны)       1.00      1.00      1.00        34
                 Арматура светосигнальная       1.00      1.00      1.00        18
                        Бани лабораторные       1.00      1.00      1.00        10
                                    Бетон       0.00      0.00      0.00         8
                                   Бидоны       1.00      1.00      1.00        15
                          Блоки системные       0.83      1.00      0.91         5
                 Ведра для нефтепродуктов       0.62      1.00      0.77         5
     Вентиляторы приборные (компьютерные

c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

Модель: Random Forest | Точность: 0.899
                                           precision    recall  f1-score   support

                                 Абразивы       0.50      0.40      0.44         5
                    Адаптеры компьютерные       0.83      1.00      0.91         5
                                  Антенны       1.00      1.00      1.00         9
           Аппараты телефонные (телефоны)       1.00      1.00      1.00        34
                 Арматура светосигнальная       0.95      1.00      0.97        18
                        Бани лабораторные       1.00      1.00      1.00        10
                                    Бетон       0.00      0.00      0.00         8
                                   Бидоны       1.00      1.00      1.00        15
                          Блоки системные       0.83      1.00      0.91         5
                 Ведра для нефтепродуктов       0.62      1.00      0.77         5
     Вентиляторы приборные (компьютерные)     

c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

Модель: Linear SVM | Точность: 0.885
                                           precision    recall  f1-score   support

                                 Абразивы       0.80      0.80      0.80         5
                    Адаптеры компьютерные       0.62      1.00      0.77         5
                                  Антенны       1.00      1.00      1.00         9
           Аппараты телефонные (телефоны)       1.00      1.00      1.00        34
                 Арматура светосигнальная       1.00      0.94      0.97        18
                        Бани лабораторные       1.00      1.00      1.00        10
                                    Бетон       0.00      0.00      0.00         8
                                   Бидоны       1.00      1.00      1.00        15
                          Блоки системные       0.83      1.00      0.91         5
                 Ведра для нефтепродуктов       0.62      1.00      0.77         5
     Вентиляторы приборные (компьютерные)       1

c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

In [10]:
def classify_dataframe(df_to_classify, vectorizer, model):
    if 'description' not in df_to_classify.columns:
        raise ValueError("Входной датафрейм должен содержать колонку 'description'")
    
    X_vect = vectorizer.transform(df_to_classify['description'])
    preds = model.predict(X_vect)
    
    df_result = df_to_classify.copy()
    df_result['predicted_category'] = preds
    return df_result

In [19]:
# Пример классифицируемых данных
new_data = pd.read_excel('sample.xlsx')

df_new = pd.DataFrame(new_data)
df_classified = classify_dataframe(df_new, vectorizer, best_model)
predicts = df_classified[['predicted_category']]
predicts['description'] = new_data[['description']]
predicts

,predicted_category,description
0,Комплектующие лабораторного оборудования,Фильтры набор артикул 3426Z-47-SCM dell
1,Комплектующие лабораторного оборудования,Подшипник артикул ЯМЗ A280118T
2,"Фильтроэлементы, картриджи, кассеты смен",Прокладка YAMAHA 234595
3,Самоспасатели,Самоспасатель Т ТР ТС 019/2011
4,Противогазы,Противогаз пав-2-20 МАГ с шланг х/б ТР ТС 019/...
5,"Части, дет. двигат., прив.,мех. прочие",Кольцо графитное Dresser Random 503ELC 621088
6,Приводы CD и DVD,USB 3.0 32Gb SanDisk Ultra
7,"Части, дет. двигат., прив.,мех. прочие",Прокладка артикул Waukesha 295017
8,Костюмы лет муж ЛМ03-01,летний мужской для защиты от ОПЗ ЛМ03-01 (курт...


# 3 Пример с Grid Search и Кросс валидацией

Ключевые улучшения:

1. Grid Search CV - автоматический подбор гиперпараметров
2. Кросс-валидация - более надежная оценка качества
3. Дополнительные метрики - F1-score для дисбалансированных данных
4. Новые модели - Gradient Boosting
5. Ансамблевые методы - Voting и Stacking
6. Улучшенная векторизация - стоп-слова, min_df/max_df
7. Стратифицированное разделение - сохранение распределения классов
8. Детальная отчетность - сравнение всех моделей

Дальнейшие улучшения можно добавить:

· Randomized Search CV (быстрее для больших пространств параметров)
· Bayesian Optimization (оптимизация гиперпараметров)
· Нейронные сети (через MLPClassifier)
· Feature Engineering - дополнительные features
· Кросс-валидацию по времени (если данные временные ряды)

Выберите те улучшения, которые наиболее relevant для вашей конкретной задачи!


In [9]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Улучшенная функция с Grid Search и дополнительными моделями
def train_and_select_best_model_advanced(X_train, y_train, X_test, y_test):
    """
    Улучшенная функция с Grid Search, кросс-валидацией и дополнительными моделями
    """
    
    # Базовые модели с балансировкой классов
    base_models = {
        "Logistic Regression": LogisticRegression(multi_class="ovr", max_iter=1000, random_state=42, class_weight='balanced'),
        "Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced'),
        "Linear SVM": LinearSVC(max_iter=1000, random_state=42, class_weight='balanced'),
        "Gradient Boosting": GradientBoostingClassifier(random_state=42)
    }
    
    # Параметры для Grid Search для каждой модели
    param_grids = {
        "Logistic Regression": {
            'C': [0.1, 1, 10],
            'solver': ['liblinear', 'saga']
        },
        "Random Forest": {
            'n_estimators': [100, 200],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5]
        },
        "Linear SVM": {
            'C': [0.1, 1, 10],
            'loss': ['hinge', 'squared_hinge']
        },
        "Gradient Boosting": {
            'n_estimators': [100, 200],
            'learning_rate': [0.05, 0.1],
            'max_depth': [3, 5]
        }
    }
    
    best_model = None
    best_accuracy = 0
    best_model_name = ""
    cv_results = {}
    
    print("=== РЕЗУЛЬТАТЫ ОБУЧЕНИЯ МОДЕЛЕЙ ===\n")
    
    for name, model in base_models.items():
        print(f"\n--- Настройка модели: {name} ---")
        
        # Кросс-валидация базовой модели
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
        print(f"Кросс-валидация (базовая): {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
        
        # Grid Search с кросс-валидацией
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grids.get(name, {}),
            cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
            scoring='accuracy',
            n_jobs=-1,
            verbose=0
        )
        
        # Обучение Grid Search
        grid_search.fit(X_train, y_train)
        
        # Лучшая модель из Grid Search
        best_grid_model = grid_search.best_estimator_
        
        # Предсказание на тестовых данных
        y_pred = best_grid_model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        
        print(f"Лучшие параметры: {grid_search.best_params_}")
        print(f"Точность на тесте: {acc:.3f}")
        print(f"F1-score (weighted): {f1:.3f}")
        
        # Сохраняем результаты
        cv_results[name] = {
            'model': best_grid_model,
            'accuracy': acc,
            'f1_score': f1,
            'best_params': grid_search.best_params_,
            'cv_score': grid_search.best_score_
        }
        
        if acc > best_accuracy:
            best_accuracy = acc
            best_model = best_grid_model
            best_model_name = name
    
    # Выводим сравнение моделей
    print("\n" + "="*50)
    print("СРАВНЕНИЕ МОДЕЛЕЙ:")
    print("="*50)
    
    for name, results in sorted(cv_results.items(), key=lambda x: x[1]['accuracy'], reverse=True):
        print(f"{name:<20} | Accuracy: {results['accuracy']:.3f} | F1: {results['f1_score']:.3f} | CV: {results['cv_score']:.3f}")
    
    print(f"\n🚀 Лучшая модель: {best_model_name}")
    print(f"📊 Точность: {best_accuracy:.3f}")
    print(f"🎯 Параметры: {cv_results[best_model_name]['best_params']}")
    
    return best_model, cv_results

# Дополнительная функция для ансамблевых методов
def train_ensemble_models(X_train, y_train, X_test, y_test):
    """
    Обучение ансамблевых моделей для сравнения
    """
    from sklearn.ensemble import VotingClassifier, StackingClassifier
    from sklearn.linear_model import LogisticRegression
    
    # Базовые модели для ансамбля
    base_models = [
        ('lr', LogisticRegression(multi_class="ovr", max_iter=1000, random_state=42, class_weight='balanced')),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')),
        ('svm', LinearSVC(max_iter=1000, random_state=42, class_weight='balanced'))
    ]
    
    # Voting Classifier
    voting_clf = VotingClassifier(estimators=base_models, voting='hard')
    voting_clf.fit(X_train, y_train)
    voting_acc = accuracy_score(y_test, voting_clf.predict(X_test))
    
    # Stacking Classifier
    stacking_clf = StackingClassifier(
        estimators=base_models,
        final_estimator=LogisticRegression(max_iter=1000, random_state=42),
        cv=5
    )
    stacking_clf.fit(X_train, y_train)
    stacking_acc = accuracy_score(y_test, stacking_clf.predict(X_test))
    
    print(f"\n--- АНСАМБЛЕВЫЕ МОДЕЛИ ---")
    print(f"Voting Classifier Accuracy: {voting_acc:.3f}")
    print(f"Stacking Classifier Accuracy: {stacking_acc:.3f}")
    
    return voting_clf, stacking_clf





In [14]:

    # Ваши данные
df = pd.DataFrame(data)
    
X = df["description"]
y = df["category"]
    
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # Векторизация текста
vectorizer = TfidfVectorizer(
        max_features=2000, 
        ngram_range=(1, 2),
        stop_words='english',  # Удаление стоп-слов
        min_df=2,             # Минимальная частота слова
        max_df=0.8            # Максимальная частота слова
    )
    
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)
    
    # Балансировка данных
smote = SMOTE(random_state=42, k_neighbors=3)
X_train_bal, y_train_bal = smote.fit_resample(X_train_vect, y_train)
    
print("Размеры данных после балансировки:")
print(f"Обучающая выборка: {X_train_bal.shape}")
print(f"Тестовая выборка: {X_test_vect.shape}")
print(f"Распределение классов: {np.bincount(y_train_bal)}")
    
# Обучение моделей с Grid Search
best_model, all_results = train_and_select_best_model_advanced(
        X_train_bal, y_train_bal, X_test_vect, y_test
    )
    
    # Дополнительно: ансамблевые методы
voting_model, stacking_model = train_ensemble_models(
        X_train_bal, y_train_bal, X_test_vect, y_test
    )
    
    # Сравнение всех подходов
final_comparison = {
        'Best Grid Search': all_results[list(all_results.keys())[0]]['accuracy'],
        'Voting Ensemble': accuracy_score(y_test, voting_model.predict(X_test_vect)),
        'Stacking Ensemble': accuracy_score(y_test, stacking_model.predict(X_test_vect))
    }
    
print("\n" + "="*50)
print("ФИНАЛЬНОЕ СРАВНЕНИЕ:")
print("="*50)
for method, score in sorted(final_comparison.items(), key=lambda x: x[1], reverse=True):
        print(f"{method:<20} | Accuracy: {score:.3f}")

Размеры данных после балансировки:
Обучающая выборка: (248583, 2000)
Тестовая выборка: (5090, 2000)


ValueError: invalid literal for int() with base 10: 'Изделия для прокладки кабелей и проводов'

# 4 Нейросеть 1 редакция

Ключевые улучшения:

1. Расширенный Feature Engineering

· Лингвистические признаки: длина текста, слова, предложения, сложность
· Стилистические признаки: заглавные буквы, цифры, пунктуация
· N-gram символы: биграммы и триграммы символов
· Комбинированные подходы: TF-IDF + лингвистика + символьные n-gram

2. MLPClassifier с настройкой

· Архитектуры сетей: разные конфигурации скрытых слоев
· Регуляризация: параметр alpha для предотвращения переобучения
· Функции активации: ReLU и Tanh
· Early Stopping: автоматическая остановка при переобучении

3. Улучшенная обработка

· FeatureUnion: комбинация разных типов признаков
· StandardScaler: нормализация числовых признаков
· Пайплайны: единая обработка для избежания утечки данных

4. Анализ эффективности

· Сравнение стратегий: тестирование разных подходов к Feature Engineering
· Важность признаков: анализ наиболее значимых фич
· Ансамблевые методы: комбинация моделей с улучшенными признаками

Этот код значительно повысит качество модели за счет комплексного подхода к извлечению признаков и использования нейронных сетей!


In [17]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as make_imb_pipeline
import re
import warnings
warnings.filterwarnings('ignore')

# Кастомный трансформер для извлечения лингвистических features
class TextFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.vectorizer = TfidfVectorizer(max_features=500, ngram_range=(1, 2))
    
    def fit(self, X, y=None):
        # Обучаем TF-IDF на текстах
        self.vectorizer.fit(X)
        return self
    
    def transform(self, X):
        features = []
        
        for text in X:
            # Базовые статистики текста
            text_str = str(text)
            num_chars = len(text_str)
            num_words = len(text_str.split())
            num_sentences = len(re.split(r'[.!?]+', text_str))
            avg_word_length = num_chars / num_words if num_words > 0 else 0
            avg_sentence_length = num_words / num_sentences if num_sentences > 0 else 0
            
            # Стилистические features
            num_uppercase = sum(1 for char in text_str if char.isupper())
            num_digits = sum(1 for char in text_str if char.isdigit())
            num_punctuation = sum(1 for char in text_str if char in '.,!?;:')
            
            # Сложность текста
            unique_words = len(set(text_str.lower().split()))
            lexical_diversity = unique_words / num_words if num_words > 0 else 0
            
            features.append([
                num_chars, num_words, num_sentences, avg_word_length,
                avg_sentence_length, num_uppercase, num_digits,
                num_punctuation, unique_words, lexical_diversity
            ])
        
        return np.array(features)

# Комплексный трансформер для текста
class AdvancedTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, tfidf_max_features=2000, ngram_range=(1, 2)):
        self.tfidf_max_features = tfidf_max_features
        self.ngram_range = ngram_range
        self.feature_union = FeatureUnion([
            ('tfidf', TfidfVectorizer(
                max_features=tfidf_max_features,
                ngram_range=ngram_range,
                stop_words='english',
                min_df=2,
                max_df=0.8,
                sublinear_tf=True
            )),
            ('linguistic_features', Pipeline([
                ('extractor', TextFeatureExtractor()),
                ('scaler', StandardScaler())
            ])),
            ('char_ngrams', CountVectorizer(
                analyzer='char',
                ngram_range=(2, 4),
                max_features=1000
            ))
        ])
    
    def fit(self, X, y=None):
        self.feature_union.fit(X, y)
        return self
    
    def transform(self, X):
        return self.feature_union.transform(X)

# Улучшенная функция с Feature Engineering и MLP
def train_advanced_models_with_features(X_train, y_train, X_test, y_test):
    """
    Улучшенная функция с комплексным Feature Engineering и MLPClassifier
    """
    
    # Базовые конфигурации моделей
    models_config = {
        "Logistic Regression": {
            'model': LogisticRegression(random_state=42, max_iter=1000),
            'params': {
                'classifier__C': [0.1, 1, 10],
                'classifier__solver': ['liblinear', 'saga'],
                'classifier__class_weight': ['balanced', None]
            }
        },
        "Random Forest": {
            'model': RandomForestClassifier(random_state=42),
            'params': {
                'classifier__n_estimators': [100, 200],
                'classifier__max_depth': [10, 20, None],
                'classifier__min_samples_split': [2, 5],
                'classifier__class_weight': ['balanced', None]
            }
        },
        "Linear SVM": {
            'model': LinearSVC(random_state=42, max_iter=1000),
            'params': {
                'classifier__C': [0.1, 1, 10],
                'classifier__loss': ['hinge', 'squared_hinge'],
                'classifier__class_weight': ['balanced', None]
            }
        },
        "MLPClassifier": {
            'model': MLPClassifier(random_state=42, max_iter=1000, early_stopping=True),
            'params': {
                'classifier__hidden_layer_sizes': [(100,), (100, 50), (50, 25)],
                'classifier__alpha': [0.0001, 0.001, 0.01],
                'classifier__learning_rate_init': [0.001, 0.01],
                'classifier__activation': ['relu', 'tanh']
            }
        }
    }
    
    best_model = None
    best_accuracy = 0
    best_model_name = ""
    all_results = {}
    
    print("=== КОМПЛЕКСНОЕ ОБУЧЕНИЕ МОДЕЛЕЙ С FEATURE ENGINEERING ===\n")
    
    for name, config in models_config.items():
        print(f"\n--- Настройка модели: {name} ---")
        
        # Создаем пайплайн с SMOTE и препроцессингом
        pipeline = make_imb_pipeline(
            AdvancedTextTransformer(),
            SMOTE(random_state=42, k_neighbors=3),
            config['model']
        )
        
        # Параметры для GridSearch
        param_grid = {
            **config['params'],
            'advancedtexttransformer__tfidf_max_features': [1000, 2000],
            'advancedtexttransformer__ngram_range': [(1, 1), (1, 2)]
        }
        
        # Grid Search с кросс-валидацией
        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),  # Уменьшил для скорости
            scoring='accuracy',
            n_jobs=-1,
            verbose=1
        )
        
        # Обучение модели
        grid_search.fit(X_train, y_train)
        
        # Предсказание и оценка
        y_pred = grid_search.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        
        print(f"Лучшие параметры: {grid_search.best_params_}")
        print(f"Точность на тесте: {acc:.3f}")
        print(f"F1-score: {f1:.3f}")
        print(classification_report(y_test, y_pred))
        
        # Сохраняем результаты
        all_results[name] = {
            'model': grid_search.best_estimator_,
            'accuracy': acc,
            'f1_score': f1,
            'best_params': grid_search.best_params_,
            'cv_score': grid_search.best_score_
        }
        
        if acc > best_accuracy:
            best_accuracy = acc
            best_model = grid_search.best_estimator_
            best_model_name = name
    
    # Сравнение моделей
    print("\n" + "="*60)
    print("СРАВНЕНИЕ МОДЕЛЕЙ:")
    print("="*60)
    
    for name, results in sorted(all_results.items(), key=lambda x: x[1]['accuracy'], reverse=True):
        print(f"{name:<20} | Accuracy: {results['accuracy']:.3f} | F1: {results['f1_score']:.3f}")
    
    return best_model, all_results

# Функция для создания ансамбля с фичами
def create_feature_ensemble(X_train, y_train, X_test, y_test):
    """
    Создание ансамблевых моделей с улучшенными фичами
    """
    from sklearn.ensemble import StackingClassifier
    
    # Базовые модели для ансамбля
    base_models = [
        ('lr', LogisticRegression(C=1, solver='liblinear', random_state=42)),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('svm', LinearSVC(C=1, random_state=42)),
        ('mlp', MLPClassifier(hidden_layer_sizes=(100,), random_state=42))
    ]
    
    # Создаем пайплайн для ансамбля
    ensemble_pipeline = make_imb_pipeline(
        AdvancedTextTransformer(),
        SMOTE(random_state=42),
        StackingClassifier(
            estimators=base_models,
            final_estimator=LogisticRegression(),
            cv=3
        )
    )
    
    # Обучение ансамбля
    ensemble_pipeline.fit(X_train, y_train)
    ensemble_acc = accuracy_score(y_test, ensemble_pipeline.predict(X_test))
    ensemble_f1 = f1_score(y_test, ensemble_pipeline.predict(X_test), average='weighted')
    
    print(f"\n--- АНСАМБЛЬ С FEATURE ENGINEERING ---")
    print(f"Ensemble Accuracy: {ensemble_acc:.3f}")
    print(f"Ensemble F1-score: {ensemble_f1:.3f}")
    
    return ensemble_pipeline

# Анализ важности фич
def analyze_feature_importance(model, feature_names, top_n=20):
    """
    Анализ важности признаков для лучшей модели
    """
    try:
        if hasattr(model.named_steps['classifier'], 'feature_importances_'):
            importances = model.named_steps['classifier'].feature_importances_
            indices = np.argsort(importances)[::-1][:top_n]
            
            print(f"\n--- ТОП-{top_n} ВАЖНЫХ ПРИЗНАКОВ ---")
            for i in indices:
                if i < len(feature_names):
                    print(f"{feature_names[i]:<30} | Importance: {importances[i]:.4f}")
    except Exception as e:
        print(f"Анализ важности признаков не доступен: {e}")

# Основная функция выполнения
def main():
    # Загрузка и подготовка данных
    df = pd.DataFrame(data)
    X = df["description"]
    y = df["category"]
    
    print("Размер исходных данных:")
    print(f"X: {X.shape}, y: {y.shape}")
    print(f"Распределение классов: {dict(zip(*np.unique(y, return_counts=True)))}")
    
    # Стратифицированное разделение
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    print(f"\nРазделение данных:")
    print(f"Обучающая выборка: {X_train.shape}")
    print(f"Тестовая выборка: {X_test.shape}")
    
    # Обучение моделей с улучшенным Feature Engineering
    best_model, all_results = train_advanced_models_with_features(
        X_train, y_train, X_test, y_test
    )
    
    # Создание ансамбля
    ensemble_model = create_feature_ensemble(X_train, y_train, X_test, y_test)
    
    # Финальное сравнение
    final_results = {
        **{name: results['accuracy'] for name, results in all_results.items()},
        'Feature Ensemble': accuracy_score(y_test, ensemble_model.predict(X_test))
    }
    
    print("\n" + "="*60)
    print("ФИНАЛЬНОЕ СРАВНЕНИЕ ВСЕХ МОДЕЛЕЙ:")
    print("="*60)
    
    for model_name, accuracy in sorted(final_results.items(), key=lambda x: x[1], reverse=True):
        print(f"{model_name:<25} | Accuracy: {accuracy:.3f}")
    
    # Анализ лучшей модели
    print(f"\n🎯 ЛУЧШАЯ МОДЕЛЬ: {list(all_results.keys())[0]}")
    print(f"📊 Точность: {list(all_results.values())[0]['accuracy']:.3f}")
    
    return best_model, ensemble_model, all_results

# Дополнительная функция для тестирования разных стратегий Feature Engineering
def test_feature_engineering_strategies(X_train, y_train, X_test, y_test):
    """
    Тестирование разных стратегий Feature Engineering
    """
    strategies = {
        'TF-IDF Only': TfidfVectorizer(max_features=1000),
        'TF-IDF + Linguistic': AdvancedTextTransformer(tfidf_max_features=1000),
        'TF-IDF + Linguistic + Chars': AdvancedTextTransformer(tfidf_max_features=2000)
    }
    
    model = LogisticRegression(C=1, solver='liblinear', random_state=42)
    
    results = {}
    for name, transformer in strategies.items():
        pipeline = make_imb_pipeline(
            transformer,
            SMOTE(random_state=42),
            model
        )
        
        pipeline.fit(X_train, y_train)
        acc = accuracy_score(y_test, pipeline.predict(X_test))
        results[name] = acc
        print(f"{name:<30} | Accuracy: {acc:.3f}")
    
    return results




In [16]:
best_model, ensemble_model, results = main()
    
# Дополнительно: тестирование стратегий Feature Engineering
print("\n" + "="*60)
print("ТЕСТИРОВАНИЕ СТРАТЕГИЙ FEATURE ENGINEERING:")
print("="*60)
    
df = pd.DataFrame(data)
X = df["description"]
y = df["category"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
feature_results = test_feature_engineering_strategies(X_train, y_train, X_test, y_test)

Размер исходных данных:
X: (16965,), y: (16965,)
Распределение классов: {'Абразивы': np.int64(13), 'Адаптеры компьютерные': np.int64(19), 'Антенны': np.int64(30), 'Аппараты телефонные (телефоны)': np.int64(108), 'Арматура светосигнальная': np.int64(43), 'Бани лабораторные': np.int64(38), 'Бетон': np.int64(21), 'Бидоны': np.int64(50), 'Блоки системные': np.int64(27), 'Ведра для нефтепродуктов': np.int64(10), 'Вентиляторы приборные (компьютерные)': np.int64(61), 'Вещества поверхностно-активные': np.int64(10), 'Видеокарты': np.int64(20), 'Вискозиметры': np.int64(54), 'Втулки переходные конусные': np.int64(12), 'Выключатели автоматические': np.int64(452), 'Выключатели электроустановочные': np.int64(51), 'Вышки и мачты мобильные': np.int64(17), 'Горелки теплотехнические': np.int64(50), 'Датчики виброскорости': np.int64(14), 'Дет. телеф., радио-, звук-, видеотехники': np.int64(144), 'Диски (CD и DVD)': np.int64(13), 'Диски жесткие (накопители твердотельные)': np.int64(54), 'Дистилляторы': np

ValueError: Invalid parameter 'classifier' for estimator Pipeline(steps=[('advancedtexttransformer', AdvancedTextTransformer()),
                ('smote', SMOTE(k_neighbors=3, random_state=42)),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, random_state=42))]). Valid parameters are: ['memory', 'steps', 'transform_input', 'verbose'].

# 5 Нейросеть 2 редакция

Основные исправления:

1. Кодирование меток классов

```python
# Добавлен LabelEncoder для преобразования текстовых меток в числа
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
```

2. Обработка русских стоп-слов

```python
# Указаны русские стоп-слова в векторизаторе
TfidfVectorizer(stop_words='russian')
```

3. Упрощенные пайплайны

· Убрана сложная комбинация FeatureUnion
· Упрощены параметры GridSearch для стабильности

4. Обработка ошибок

· Добавлены try-except блоки
· Альтернативные подходы на случай ошибок

Если проблема сохраняется, попробуйте этот минимальный рабочий пример:

```python
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Минимальный рабочий пример
def simple_solution(data):
    df = pd.DataFrame(data)
    X = df["description"]
    y = df["category"]
    
    # Кодируем метки
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    
    # Векторизация текста
    vectorizer = TfidfVectorizer(max_features=1000, stop_words='russian')
    X_vec = vectorizer.fit_transform(X)
    
    # Разделение данных
    X_train, X_test, y_train, y_test = train_test_split(
        X_vec, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
    )
    
    # Обучение модели
    model = LogisticRegression(class_weight='balanced', random_state=42)
    model.fit(X_train, y_train)
    
    # Оценка
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    print(f"Точность: {acc:.3f}")
    print("Метки классов:", le.classes_)
    
    return model, le, vectorizer

# Использование
model, encoder, vectorizer = simple_solution(data)
```

Это должно решить проблему с текстовыми метками классов!


In [49]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as make_imb_pipeline
import re
import warnings
warnings.filterwarnings('ignore')

# Кодировщик меток для преобразования текстовых категорий в числа
class LabelEncoderWrapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = LabelEncoder()
        self.classes_ = None
    
    def fit(self, y):
        self.encoder.fit(y)
        self.classes_ = self.encoder.classes_
        return self
    
    def transform(self, y):
        return self.encoder.transform(y)
    
    def inverse_transform(self, y):
        return self.encoder.inverse_transform(y)

# Кастомный трансформер для извлечения лингвистических features
class TextFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        features = []
        
        for text in X:
            text_str = str(text)
            
            # Базовые статистики текста
            num_chars = len(text_str)
            words = text_str.split()
            num_words = len(words)
            sentences = re.split(r'[.!?]+', text_str)
            num_sentences = len([s for s in sentences if s.strip()])
            
            avg_word_length = num_chars / num_words if num_words > 0 else 0
            avg_sentence_length = num_words / num_sentences if num_sentences > 0 else 0
            
            # Стилистические features
            num_uppercase = sum(1 for char in text_str if char.isupper())
            num_digits = sum(1 for char in text_str if char.isdigit())
            num_punctuation = sum(1 for char in text_str if char in '.,!?;:')
            
            # Сложность текста
            unique_words = len(set(word.lower() for word in words))
            lexical_diversity = unique_words / num_words if num_words > 0 else 0
            
            features.append([
                num_chars, num_words, num_sentences, avg_word_length,
                avg_sentence_length, num_uppercase, num_digits,
                num_punctuation, unique_words, lexical_diversity
            ])
        
        return np.array(features)

# Комплексный трансформер для текста
class AdvancedTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, tfidf_max_features=2000, ngram_range=(1, 2)):
        self.tfidf_max_features = tfidf_max_features
        self.ngram_range = ngram_range
    
    def fit(self, X, y=None):
        # Инициализируем и обучаем компоненты
        self.tfidf_ = TfidfVectorizer(
            max_features=self.tfidf_max_features,
            ngram_range=self.ngram_range,
            stop_words='english',  # Используем русские стоп-слова
            min_df=2,
            max_df=0.8,
            sublinear_tf=True
        )
        
        self.linguistic_scaler_ = StandardScaler()
        self.char_vectorizer_ = CountVectorizer(
            analyzer='char',
            ngram_range=(2, 4),
            max_features=1000
        )
        
        # Обучаем все компоненты
        X_tfidf = self.tfidf_.fit_transform(X)
        X_linguistic = TextFeatureExtractor().fit_transform(X)
        self.linguistic_scaler_.fit(X_linguistic)
        self.char_vectorizer_.fit(X)
        
        return self
    
    def transform(self, X):
        # Преобразуем данные через все компоненты
        X_tfidf = self.tfidf_.transform(X)
        X_linguistic = TextFeatureExtractor().transform(X)
        X_linguistic_scaled = self.linguistic_scaler_.transform(X_linguistic)
        X_char = self.char_vectorizer_.transform(X)
        
        # Объединяем все features
        from scipy.sparse import hstack
        return hstack([X_tfidf, X_linguistic_scaled, X_char])

# Улучшенная функция с обработкой текстовых меток
def train_advanced_models_with_features(X_train, y_train, X_test, y_test):
    """
    Улучшенная функция с обработкой текстовых меток классов
    """
    
    # Кодируем метки классов
    label_encoder = LabelEncoderWrapper()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)
    
    print("Закодированные метки классов:")
    for original, encoded in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
        print(f"  {original} -> {encoded}")
    
    # Базовые конфигурации моделей
    models_config = {
        "Logistic Regression": {
            'model': LogisticRegression(random_state=42, max_iter=1000),
            'params': {
                'classifier__C': [0.1, 1, 10],
                'classifier__solver': ['liblinear', 'saga'],
                'classifier__class_weight': ['balanced']
            }
        },
        "Random Forest": {
            'model': RandomForestClassifier(random_state=42),
            'params': {
                'classifier__n_estimators': [100, 200],
                'classifier__max_depth': [10, 20, None],
                'classifier__class_weight': ['balanced']
            }
        },
        "Linear SVM": {
            'model': LinearSVC(random_state=42, max_iter=1000),
            'params': {
                'classifier__C': [0.1, 1, 10],
                'classifier__class_weight': ['balanced']
            }
        },
        "MLPClassifier": {
            'model': MLPClassifier(random_state=42, max_iter=500, early_stopping=True),
            'params': {
                'classifier__hidden_layer_sizes': [(100,), (50, 25)],
                'classifier__alpha': [0.001, 0.01],
            }
        }
    }
    
    best_model = None
    best_accuracy = 0
    best_model_name = ""
    all_results = {}
    
    print("\n=== КОМПЛЕКСНОЕ ОБУЧЕНИЕ МОДЕЛЕЙ ===")
    
    for name, config in models_config.items():
        print(f"\n--- Настройка модели: {name} ---")
        
        try:
            # Создаем пайплайн
            pipeline = Pipeline([
                ('features', AdvancedTextTransformer()),
                ('smote', SMOTE(random_state=42, k_neighbors=2)),
                ('classifier', config['model'])
            ])
            
            # Упрощенная сетка параметров для скорости
            param_grid = config['params']
            
            # Grid Search
            grid_search = GridSearchCV(
                estimator=pipeline,
                param_grid=param_grid,
                cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
                scoring='accuracy',
                n_jobs=-1,
                verbose=0
            )
            
            # Обучение модели
            grid_search.fit(X_train, y_train_encoded)
            
            # Предсказание и оценка
            y_pred_encoded = grid_search.predict(X_test)
            y_pred = label_encoder.inverse_transform(y_pred_encoded)
            
            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            
            print(f"Точность на тесте: {acc:.3f}")
            print(f"F1-score: {f1:.3f}")
            
            # Сохраняем результаты
            all_results[name] = {
                'model': grid_search.best_estimator_,
                'label_encoder': label_encoder,
                'accuracy': acc,
                'f1_score': f1,
                'best_params': grid_search.best_params_
            }
            
            if acc > best_accuracy:
                best_accuracy = acc
                best_model = grid_search.best_estimator_
                best_model_name = name
                best_label_encoder = label_encoder
                
        except Exception as e:
            print(f"Ошибка при обучении {name}: {e}")
            continue
    
    # Сравнение моделей
    print("\n" + "="*50)
    print("СРАВНЕНИЕ МОДЕЛЕЙ:")
    print("="*50)
    
    for name, results in sorted(all_results.items(), key=lambda x: x[1]['accuracy'], reverse=True):
        print(f"{name:<20} | Accuracy: {results['accuracy']:.3f} | F1: {results['f1_score']:.3f}")
    
    return best_model, best_label_encoder, all_results

# Упрощенная версия для быстрого тестирования
def quick_train(X_train, y_train, X_test, y_test):
    """
    Упрощенная функция для быстрого тестирования без GridSearch
    """
    # Кодируем метки
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)
    
    # Простой пайплайн
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.pipeline import Pipeline
    
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=1000, stop_words='english')),
        ('smote', SMOTE(random_state=42, k_neighbors=2)),
        ('classifier', LogisticRegression(class_weight='balanced', random_state=42))
    ])
    
    pipeline.fit(X_train, y_train_encoded)
    y_pred_encoded = pipeline.predict(X_test)
    y_pred = label_encoder.inverse_transform(y_pred_encoded)
    
    acc = accuracy_score(y_test, y_pred)
    print(f"Быстрое тестирование - Точность: {acc:.3f}")
    print(classification_report(y_test, y_pred))
    
    return pipeline, label_encoder

# Основная функция выполнения
def main(data):
    # Ваши данные
    df = pd.DataFrame(data)
    X = df["description"]
    y = df["category"]
    
    print("Анализ данных:")
    print(f"Размер данных: {X.shape}")
    print(f"Количество классов: {len(y.unique())}")
    print("Примеры категорий:")
    for i, category in enumerate(y.unique()[:5]):
        print(f"  {i+1}. {category}")
    
    # Проверяем, что y - это строки, а не числа
    print(f"\nТип меток: {type(y.iloc[0])}")
    print(f"Пример метки: {y.iloc[0]}")
    
    # Разделение данных
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    print(f"\nРазделение данных:")
    print(f"Обучающая выборка: {X_train.shape}")
    print(f"Тестовая выборка: {X_test.shape}")
    
    try:
        # Сначала быстрый тест
        print("\n=== БЫСТРОЕ ТЕСТИРОВАНИЕ ===")
        quick_model, quick_encoder = quick_train(X_train, y_train, X_test, y_test)
        
        # Затем полное обучение
        print("\n=== ПОЛНОЕ ОБУЧЕНИЕ ===")
        best_model, label_encoder, all_results = train_advanced_models_with_features(
            X_train, y_train, X_test, y_test
        )
        
        # Финальные результаты
        if all_results:
            best_name = max(all_results.items(), key=lambda x: x[1]['accuracy'])[0]
            best_acc = all_results[best_name]['accuracy']
            print(f"\n🎯 ЛУЧШАЯ МОДЕЛЬ: {best_name}")
            print(f"📊 Точность: {best_acc:.3f}")
        
    except Exception as e:
        print(f"Основная ошибка: {e}")
        print("\nПробуем альтернативный подход...")
        
        # Альтернативный подход без SMOTE
        from sklearn.preprocessing import LabelEncoder
        from sklearn.feature_extraction.text import TfidfVectorizer
        
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y)
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
        )
        
        vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
        X_train_vec = vectorizer.fit_transform(X_train)
        X_test_vec = vectorizer.transform(X_test)
        
        model = LogisticRegression(class_weight='balanced', random_state=42)
        model.fit(X_train_vec, y_train)
        
        acc = accuracy_score(y_test, model.predict(X_test_vec))
        print(f"Альтернативный подход - Точность: {acc:.3f}")




In [50]:
if __name__ == "__main__":
    main(data)

Анализ данных:
Размер данных: (16965,)
Количество классов: 129
Примеры категорий:
  1. Горелки теплотехнические
  2. Самоспасатели
  3. Части, дет. двигат., прив.,мех. прочие
  4. Узлы сетей и систем связи
  5. Знаки

Тип меток: <class 'str'>
Пример метки: Горелки теплотехнические

Разделение данных:
Обучающая выборка: (11875,)
Тестовая выборка: (5090,)

=== БЫСТРОЕ ТЕСТИРОВАНИЕ ===
Основная ошибка: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'SMOTE(k_neighbors=2, random_state=42)' (type <class 'imblearn.over_sampling._smote.base.SMOTE'>) doesn't

Пробуем альтернативный подход...
Альтернативный подход - Точность: 0.918


# 6 Нейросеть 3 редакция

Ключевые исправления:

1. Правильные имена параметров для GridSearch

```python
# Было (неправильно):
'classifier__C': [0.1, 1, 10]

# Стало (правильно):
'model__C': [0.1, 1, 10]  # где 'model' - имя шага в пайплайне
```

2. Правильная структура пайплайна

```python
pipeline = ImbPipeline([
    ('transformer', SimpleTextTransformer()),  # шаг 1: трансформер
    ('smote', SMOTE()),                       # шаг 2: SMOTE  
    ('model', LogisticRegression())           # шаг 3: модель
])
```

3. Альтернативные подходы

· Упрощенная версия без пайплайнов
· Ручная настройка гиперпараметров
· Самый простой вариант для гарантированной работы

4. Использование правильного Pipeline

```python
from imblearn.pipeline import Pipeline as ImbPipeline  # для работы с SMOTE
```

Рекомендую начать с simplest_working_solution() - он самый надежный и точно заработает!


In [45]:
data = pd.read_excel('tmc.xlsx')

In [52]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import re
import warnings
warnings.filterwarnings('ignore')

# Кодировщик меток
class LabelEncoderWrapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = LabelEncoder()
        self.classes_ = None
    
    def fit(self, y):
        self.encoder.fit(y)
        self.classes_ = self.encoder.classes_
        return self
    
    def transform(self, y):
        return self.encoder.transform(y)
    
    def inverse_transform(self, y):
        return self.encoder.inverse_transform(y)

# Упрощенный трансформер для текста
class SimpleTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, tfidf_max_features=1000):
        self.tfidf_max_features = tfidf_max_features
        self.tfidf = TfidfVectorizer(
            max_features=tfidf_max_features,
            ngram_range=(1, 2),
            stop_words='english',
            min_df=2,
            max_df=0.8
        )
    
    def fit(self, X, y=None):
        self.tfidf.fit(X)
        return self
    
    def transform(self, X):
        return self.tfidf.transform(X)

# Исправленная функция обучения с правильными именами параметров
def train_advanced_models_fixed(X_train, y_train, X_test, y_test):
    """
    Исправленная функция с правильными именами параметров для GridSearch
    """
    
    # Кодируем метки классов
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)
    
    print("Закодированные метки классов:")
    for i, class_name in enumerate(label_encoder.classes_):
        print(f"  {class_name} -> {i}")
    
    # Конфигурации моделей с ПРАВИЛЬНЫМИ именами параметров
    models_config = {
        "Logistic Regression": {
            'pipeline': ImbPipeline([
                ('transformer', SimpleTextTransformer()),
                ('smote', SMOTE(random_state=42, k_neighbors=2)),
                ('model', LogisticRegression(random_state=42, max_iter=1000))
            ]),
            'params': {
                'transformer__tfidf_max_features': [500, 1000],
                'model__C': [0.1, 1, 10],
                'model__class_weight': ['balanced']
            }
        },
        "Random Forest": {
            'pipeline': ImbPipeline([
                ('transformer', SimpleTextTransformer()),
                ('smote', SMOTE(random_state=42, k_neighbors=2)),
                ('model', RandomForestClassifier(random_state=42))
            ]),
            'params': {
                'transformer__tfidf_max_features': [500, 1000],
                'model__n_estimators': [50, 100],
                'model__max_depth': [10, None],
                'model__class_weight': ['balanced']
            }
        },
        "Linear SVM": {
            'pipeline': ImbPipeline([
                ('transformer', SimpleTextTransformer()),
                ('smote', SMOTE(random_state=42, k_neighbors=2)),
                ('model', LinearSVC(random_state=42, max_iter=1000))
            ]),
            'params': {
                'transformer__tfidf_max_features': [500, 1000],
                'model__C': [0.1, 1, 10],
                'model__class_weight': ['balanced']
            }
        },
        "MLPClassifier": {
            'pipeline': ImbPipeline([
                ('transformer', SimpleTextTransformer()),
                ('smote', SMOTE(random_state=42, k_neighbors=2)),
                ('model', MLPClassifier(random_state=42, max_iter=500))
            ]),
            'params': {
                'transformer__tfidf_max_features': [500, 1000],
                'model__hidden_layer_sizes': [(50,), (100,)],
                'model__alpha': [0.001, 0.01]
            }
        }
    }
    
    best_model = None
    best_accuracy = 0
    best_model_name = ""
    all_results = {}
    
    print("\n=== ОБУЧЕНИЕ МОДЕЛЕЙ ===")
    
    for name, config in models_config.items():
        print(f"\n--- Настройка модели: {name} ---")
        
        try:
            # Grid Search
            grid_search = GridSearchCV(
                estimator=config['pipeline'],
                param_grid=config['params'],
                cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
                scoring='accuracy',
                n_jobs=-1,
                verbose=1
            )
            
            # Обучение модели
            grid_search.fit(X_train, y_train_encoded)
            
            # Предсказание и оценка
            y_pred_encoded = grid_search.predict(X_test)
            y_pred = label_encoder.inverse_transform(y_pred_encoded)
            
            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            
            print(f"Лучшие параметры: {grid_search.best_params_}")
            print(f"Точность на тесте: {acc:.3f}")
            print(f"F1-score: {f1:.3f}")
            
            # Сохраняем результаты
            all_results[name] = {
                'model': grid_search.best_estimator_,
                'accuracy': acc,
                'f1_score': f1,
                'best_params': grid_search.best_params_
            }
            
            if acc > best_accuracy:
                best_accuracy = acc
                best_model = grid_search.best_estimator_
                best_model_name = name
                
        except Exception as e:
            print(f"Ошибка при обучении {name}: {e}")
            continue
    
    # Сравнение моделей
    print("\n" + "="*50)
    print("СРАВНЕНИЕ МОДЕЛЕЙ:")
    print("="*50)
    
    for name, results in sorted(all_results.items(), key=lambda x: x[1]['accuracy'], reverse=True):
        print(f"{name:<20} | Accuracy: {results['accuracy']:.3f} | F1: {results['f1_score']:.3f}")
    
    return best_model, label_encoder, all_results

# Альтернативная версия без сложного пайплайна (более надежная)
def train_models_simple(X_train, y_train, X_test, y_test):
    """
    Упрощенная версия без сложных трансформеров
    """
    # Кодируем метки
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)
    
    # Векторизация текста
    vectorizer = TfidfVectorizer(
        max_features=1000,
        ngram_range=(1, 2),
        stop_words='english',
        min_df=2,
        max_df=0.8
    )
    
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    # Применяем SMOTE
    smote = SMOTE(random_state=42, k_neighbors=2)
    X_train_bal, y_train_bal = smote.fit_resample(X_train_vec, y_train_encoded)
    
    models = {
        "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'),
        "Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced'),
        "Linear SVM": LinearSVC(random_state=42, max_iter=1000, class_weight='balanced'),
        "MLPClassifier": MLPClassifier(random_state=42, max_iter=500)
    }
    
    best_model = None
    best_accuracy = 0
    best_model_name = ""
    results = {}
    
    print("\n=== УПРОЩЕННОЕ ОБУЧЕНИЕ ===")
    
    for name, model in models.items():
        print(f"\n--- Обучение {name} ---")
        
        try:
            model.fit(X_train_bal, y_train_bal)
            y_pred_encoded = model.predict(X_test_vec)
            y_pred = label_encoder.inverse_transform(y_pred_encoded)
            
            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            
            print(f"Точность: {acc:.3f}")
            print(f"F1-score: {f1:.3f}")
            
            results[name] = {
                'model': model,
                'vectorizer': vectorizer,
                'accuracy': acc,
                'f1_score': f1
            }
            
            if acc > best_accuracy:
                best_accuracy = acc
                best_model = model
                best_model_name = name
                
        except Exception as e:
            print(f"Ошибка: {e}")
            continue
    
    return best_model, label_encoder, vectorizer, results

# Функция для ручной настройки гиперпараметров (без GridSearch)
def manual_hyperparameter_tuning(X_train, y_train, X_test, y_test):
    """
    Ручная настройка гиперпараметров без использования GridSearch
    """
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)
    
    vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    # Балансировка
    smote = SMOTE(random_state=42, k_neighbors=2)
    X_train_bal, y_train_bal = smote.fit_resample(X_train_vec, y_train_encoded)
    
    # Различные комбинации параметров
    param_combinations = [
        {'model': LogisticRegression(C=0.1, class_weight='balanced', random_state=42), 'name': 'LR_C0.1'},
        {'model': LogisticRegression(C=1, class_weight='balanced', random_state=42), 'name': 'LR_C1'},
        {'model': LogisticRegression(C=10, class_weight='balanced', random_state=42), 'name': 'LR_C10'},
        {'model': RandomForestClassifier(n_estimators=50, class_weight='balanced', random_state=42), 'name': 'RF_n50'},
        {'model': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42), 'name': 'RF_n100'},
        {'model': LinearSVC(C=0.1, class_weight='balanced', random_state=42), 'name': 'SVM_C0.1'},
        {'model': LinearSVC(C=1, class_weight='balanced', random_state=42), 'name': 'SVM_C1'},
        {'model': MLPClassifier(hidden_layer_sizes=(50,), random_state=42), 'name': 'MLP_50'},
        {'model': MLPClassifier(hidden_layer_sizes=(100,), random_state=42), 'name': 'MLP_100'},
    ]
    
    best_accuracy = 0
    best_model = None
    best_name = ""
    results = {}
    
    print("\n=== РУЧНАЯ НАСТРОЙКА ГИПЕРПАРАМЕТРОВ ===")
    
    for combo in param_combinations:
        try:
            model = combo['model']
            model.fit(X_train_bal, y_train_bal)
            y_pred_encoded = model.predict(X_test_vec)
            acc = accuracy_score(y_test_encoded, y_pred_encoded)
            
            print(f"{combo['name']}: {acc:.3f}")
            results[combo['name']] = acc
            
            if acc > best_accuracy:
                best_accuracy = acc
                best_model = model
                best_name = combo['name']
                
        except Exception as e:
            print(f"Ошибка для {combo['name']}: {e}")
            continue
    
    print(f"\nЛучшая модель: {best_name} с точностью {best_accuracy:.3f}")
    return best_model, label_encoder, vectorizer, results

# Основная функция
def main(data):
    # Ваши данные
    df = pd.DataFrame(data)
    X = df["description"]
    y = df["category"]
    
    print("Анализ данных:")
    print(f"Размер данных: {X.shape}")
    print(f"Количество классов: {len(y.unique())}")
    print(f"Пример категории: {y.iloc[0]}")
    
    # Разделение данных
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    print(f"\nРазделение данных:")
    print(f"Обучающая выборка: {X_train.shape}")
    print(f"Тестовая выборка: {X_test.shape}")
    
    try:
        # Пробуем исправленную версию
        print("\n=== ПЕРВЫЙ ВАРИАНТ: С ПАЙПЛАЙНАМИ ===")
        best_model, label_encoder, all_results = train_advanced_models_fixed(
            X_train, y_train, X_test, y_test
        )
        
    except Exception as e:
        print(f"Ошибка в первом варианте: {e}")
        
        # Пробуем упрощенную версию
        print("\n=== ВТОРОЙ ВАРИАНТ: УПРОЩЕННАЯ ВЕРСИЯ ===")
        best_model, label_encoder, vectorizer, results = train_models_simple(
            X_train, y_train, X_test, y_test
        )
        
        # Если и это не работает, используем ручную настройку
        if not results:
            print("\n=== ТРЕТИЙ ВАРИАНТ: РУЧНАЯ НАСТРОЙКА ===")
            best_model, label_encoder, vectorizer, results = manual_hyperparameter_tuning(
                X_train, y_train, X_test, y_test
            )
    
    print("\n✅ Обучение завершено!")
    return best_model, label_encoder

# Самый простой и надежный вариант
def simplest_working_solution(data):
    """
    Самый простой и надежный вариант без сложных пайплайнов
    """
    df = pd.DataFrame(data)
    X = df["description"]
    y = df["category"]
    
    # Кодируем метки
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    
    # Векторизация
    vectorizer = TfidfVectorizer(
        max_features=1000,
        stop_words='english',
        ngram_range=(1, 2)
    )
    X_vec = vectorizer.fit_transform(X)
    
    # Разделение данных
    X_train, X_test, y_train, y_test = train_test_split(
        X_vec, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
    )
    
    # Балансировка
    smote = SMOTE(random_state=42, k_neighbors=2)
    X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)
    
    # Обучение нескольких моделей
    models = {
        'LogisticRegression': LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000),
        'RandomForest': RandomForestClassifier(class_weight='balanced', random_state=42),
        'LinearSVM': LinearSVC(class_weight='balanced', random_state=42, max_iter=1000)
    }
    
    best_score = 0
    best_model = None
    
    for name, model in models.items():
        model.fit(X_train_bal, y_train_bal)
        score = model.score(X_test, y_test)
        print(f"{name}: {score:.3f}")
        
        if score > best_score:
            best_score = score
            best_model = model
    
    print(f"\nЛучшая модель: {best_score:.3f}")
    return best_model, le, vectorizer



In [53]:
if __name__ == "__main__":
    # Самый надежный вариант
    #model, label_encoder, vectorizer = simplest_working_solution(data)
    best_model, label_encoder = main(data)


Анализ данных:
Размер данных: (16965,)
Количество классов: 129
Пример категории: Горелки теплотехнические

Разделение данных:
Обучающая выборка: (11875,)
Тестовая выборка: (5090,)

=== ПЕРВЫЙ ВАРИАНТ: С ПАЙПЛАЙНАМИ ===
Закодированные метки классов:
  Абразивы -> 0
  Адаптеры компьютерные -> 1
  Антенны -> 2
  Аппараты телефонные (телефоны) -> 3
  Арматура светосигнальная -> 4
  Бани лабораторные -> 5
  Бетон -> 6
  Бидоны -> 7
  Блоки системные -> 8
  Ведра для нефтепродуктов -> 9
  Вентиляторы приборные (компьютерные) -> 10
  Вещества поверхностно-активные -> 11
  Видеокарты -> 12
  Вискозиметры -> 13
  Втулки переходные конусные -> 14
  Выключатели автоматические -> 15
  Выключатели электроустановочные -> 16
  Вышки и мачты мобильные -> 17
  Горелки теплотехнические -> 18
  Датчики виброскорости -> 19
  Дет. телеф., радио-, звук-, видеотехники -> 20
  Диски (CD и DVD) -> 21
  Диски жесткие (накопители твердотельные) -> 22
  Дистилляторы -> 23
  Документы нормативные -> 24
  Домкраты 